## Setup

In [1]:
import torch
from transformers import BertForSequenceClassification, BertTokenizerFast
import pandas as pd
import numpy as np

import random
import os
from typing import Union, List, Dict
import argparse
import pdb

In [2]:
# running option
class Config():
    seed = 0
    model = 'best'

args = Config()

In [3]:
data_path: str = './data/inference_sampleset.csv'
vocab_path: str = './vocab'
result_path: str = './result'

last_model: str = './model/checkpoint-12300/'
best_model: str = './model/checkpoint-984/'

In [4]:
# set seed for reproduction
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
set_seed(args.seed)

## Build Dataset

In [5]:
df = pd.read_csv(data_path, encoding='cp949')

# possible_labels = ['가설 설정', '기술 정의', '기술동향','기술의 파급효과', 
#                    '기술의 필요성', '대상 데이터', '데이터처리', '문제 정의', 
#                    '성능/효과', '시장동향', '이론/모형', '제안 방법', '후속연구']
possible_labels = ['성능/효과', '제안 방법', '대상 데이터', '문제 정의', 
                   '이론/모형', '후속연구', '기술 정의','데이터처리', '가설 설정', 
                   '시장동향', '기술의 파급효과', '기술동향', '기술의 필요성']

label_dict = {}
label2id = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
    label2id[index] = possible_label

## Build Model

In [6]:
tokenizer = BertTokenizerFast.from_pretrained(vocab_path, do_lower_case=False, model_max_length=128)
model_path = best_model if args.model == 'best' else last_model
model = BertForSequenceClassification.from_pretrained(model_path, 
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

## Sentence Classification with TechBERT

In [7]:
inputs = tokenizer(list(df.text), add_special_tokens=True, truncation=True, padding=True, max_length = 128, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
    
    predicted_class_id = logits.argmax(dim=1).numpy()
    prediction = [label2id[i] for i in predicted_class_id]

In [8]:
compare_df = pd.DataFrame({'text':df.text,'true':df.tag,'pred':prediction})
if os.path.isdir(result_path): pass
else: os.mkdir(result_path)
file_path = os.path.join(result_path, f'classification_result.csv')
if os.path.exists(file_path): os.remove(file_path)
compare_df.to_csv(file_path, index=False, encoding="utf-8-sig")
compare_df

,text,true,pred
0,데이터의 유출 및 손실 걱정 없는 안심 사회 구현,기술의 파급효과,기술의 파급효과
1,"전문재배사, 소프트웨어 개발자, 사물인터넷 서비스 기업 등 청년 일자리 창출",기술의 파급효과,기술의 필요성
2,본 기술이 적용된 K-FC 시스템이 도입이 되는 경우 국내 비면허 주파수 이용 환경...,기술의 파급효과,기술의 파급효과
3,"7주간의 순식 타이치 31개 동작이 간호대학생의 피로, 불안및 수면양상에 미치는 효...",성능/효과,성능/효과
4,"사진 감광막 두께차이와 감광막 제거공정만으로 실리콘 트랜치를 형성하였으며, 실제 공...",성능/효과,성능/효과
5,"일반적 특성에 따라 개인정보 보호에 대한 지식과 인식, 행위에 대한 실천정도를 분석...",성능/효과,성능/효과
6,첫째 연료 레일의 압력을 1.5bar에서 6 bar까지 변화시키며 압력과 구동 속도...,제안 방법,제안 방법
7,EEG 센서 모듈의 설계와 제작을 본 연구실에서 직접 하였으며 PC 기반의 컨트롤 ...,제안 방법,제안 방법
8,본 논문에서는 정책설정에 사용되는 대표적인 프로토콜인 SNMP와 COPS을 이용한 ...,제안 방법,제안 방법
9,"조사지역은 당귀 주산단지인 충북 제천과 경북 봉화를 대상으로 하였으며, 조사는 11...",대상 데이터,대상 데이터


In [9]:
print(f'acc: {sum(df.tag == prediction) / len(df)}')

acc: 0.9487179487179487
